# Data Processing script v2 for the SWEML v2.0
This .ipynb script uses python module for processing predownloaded NASA ASO observations by Water Year, locating nearest SNOTEL sites, connecting SNOTEL obs with ASO obs, and add geospatial features to the ML training/testing/hindcast dataframes.

#8.1.3 #7.7.3 sometimes you have to change package versions for the progress bars to work, annoying!


In [12]:
import os
HOME = os.getcwd()
import model_Domain
HOME = os.getcwd()


#If you get a proj.db error below, run the following and put the following into the terminal
import pyproj
# Get the PROJ data directory
proj_data_dir = pyproj.datadir.get_data_dir()
proj_db_path = proj_data_dir + "/proj.db"
os.environ['PROJ_LIB'] =pyproj.datadir.get_data_dir()
os.environ['PROJ_LIB']
print(proj_db_path)

#make SWEMLv2.0 modeling domain for western USA
WY_list = ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'] #'2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024' - Trying the first bit to prove concept, then can expand
output_res = 500 #desired spatial resulution in meters (m)

/uufs/chpc.utah.edu/common/home/u1154915/.conda/envs/SWEML_310/lib/python3.10/site-packages/pyproj/proj_dir/share/proj/proj.db


In [13]:
ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()
from ASOget import ASODataProcessing_v2

# Inputs for fetching ASO data for a region
short_name = 'ASO_50M_SWE'
directory = "Raw_ASO_Data"

#Get ASO data, sometime sites will give error and break code, most times you can just rerun it using the data_processor sections below (e.g., comment out other parts
for WY in WY_list:
    #Convert ASO tifs to parquet
    print(f"Converting ASO images for WY: {WY}")
    folder_name = f"{WY}/{directory}"
    data_processor = ASODataProcessing_v2() #note, 2019-5-1, 2019-06-11 seems to be bad, manually removed from SW region
    data_processor.convert_tiff_to_parquet_multiprocess(folder_name, output_res, WY) 

Converting ASO images for WY: 2013
Converting .tif to parquet
Converting 6 ASO tif files to parquet


  0%|          | 0/6 [00:00<?, ?it/s]

Checking to make sure all files successfully converted...


  0%|          | 0/6 [00:00<?, ?it/s]

Converting ASO images for WY: 2014
Converting .tif to parquet
Converting 18 ASO tif files to parquet


...

  0%|          | 0/71 [00:00<?, ?it/s]

An error occurred: Read or write failed. /uufs/chpc.utah.edu/common/home/civil-group1/Johnson/SWEMLv2.0/data/ASO/2024/Processed_500M_SWE/ASO_ASO_500M_20240411.tif, band 1: IReadBlock failed at X offset 0, Y offset 0: TIFFReadEncodedStrip() failed.


ERROR 1: TIFFReadEncodedStrip:Read error at scanline 4294967295; got 7712 bytes, expected 7904
ERROR 1: TIFFReadEncodedStrip() failed.
ERROR 1: /uufs/chpc.utah.edu/common/home/civil-group1/Johnson/SWEMLv2.0/data/ASO/2024/Processed_500M_SWE/ASO_ASO_500M_20240411.tif, band 1: IReadBlock failed at X offset 0, Y offset 0: TIFFReadEncodedStrip() failed.


Checking to make sure all files successfully converted...


  0%|          | 0/70 [00:00<?, ?it/s]

## Get Snotel and CDEC in situ observations
- clean in situ observations, specifically the CDEC sites, need a data processing method to remove outtliers and nan/0 obs
- Ideas - add nearest sites elevation, distance from cell, then can bypass sites with bad data. 

In [2]:
# Only needed once. Other spatial resolutions can use the same data

ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()
#Get in situ observations
import get_InSitu_obs
import numpy as np

#make a list of dates to align with the ASO observations (they go as early as Jan-29 and as far out as the July-17)
years = np.arange(2013,2025,1)#Needs to go 1yr out
start_month_day = '10-01'
end_month_day = '08-31'
#datelist = get_InSitu_obs.make_dates(years, start_month_day, end_month_day, WY = True)

# observations 
get_InSitu_obs.Get_Monitoring_Data_Threaded_Updated(years, start_month_day, end_month_day, WY = True)

#combine years
get_InSitu_obs.combine_dfs(years)

getting in situ snow obs metadata
Getting SNOTEL and CDEC observations for 2013
Getting California Data Exchange Center SWE data from 130 sites...


  0%|          | 0/130 [00:00<?, ?it/s]

Start retrieving data for Farewell Gap, FRW
https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/FRW:CA:MSNT%257Cid=%2522%2522%257Cname/2012-10-01,2013-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Bonanza King, BNK
https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/BNK:CA:MSNT%257Cid=%2522%2522%257Cname/2012-10-01,2013-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Cedar Pass, CDP
https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/CDP:CA:MSNT%257Cid=%2522%2522%257Cname/2012-10-01,2013-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Charlotte Lake, CRL
https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/CRL:CA:MSNT%257Cid=%2522%2522%257Cname/2012-10-01,2013-08-31/WTEQ::value?fitToS

  0%|          | 0/839 [00:00<?, ?it/s]

Start retrieving data for Adin Mtn, 301_CA_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/301:CA:SNTL%7Cid=%22%22%7Cname/2012-10-01,2013-08-31/WTEQ::value?fitToScreen=falseStart retrieving data for Agua Canyon, 907_UT_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/907:UT:SNTL%7Cid=%22%22%7Cname/2012-10-01,2013-08-31/WTEQ::value?fitToScreen=false

Start retrieving data for Albro Lake, 916_MT_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/916:MT:SNTL%7Cid=%22%22%7Cname/2012-10-01,2013-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Alexander Lake, 1267_AK_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/1267:AK:SNTL%7Cid=%22%22%7Cname/2012-10-01,2013-08-

...

Start retrieving data for Adin Mtn, 301_CA_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/301:CA:SNTL%7Cid=%22%22%7Cname/2022-10-01,2023-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Agua Canyon, 907_UT_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/907:UT:SNTL%7Cid=%22%22%7Cname/2022-10-01,2023-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Albro Lake, 916_MT_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/916:MT:SNTL%7Cid=%22%22%7Cname/2022-10-01,2023-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Alexander Lake, 1267_AK_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/1267:AK:SNTL%7Cid=%22%22%7Cname/2022-10-01,2023-08-

  0%|          | 0/130 [00:00<?, ?it/s]

Start retrieving data for Farewell Gap, FRWStart retrieving data for Bonanza King, BNK
https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/BNK:CA:MSNT%257Cid=%2522%2522%257Cname/2023-10-01,2024-08-31/WTEQ::value?fitToScreen=false

https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/FRW:CA:MSNT%257Cid=%2522%2522%257Cname/2023-10-01,2024-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Cedar Pass, CDP
https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/CDP:CA:MSNT%257Cid=%2522%2522%257Cname/2023-10-01,2024-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Charlotte Lake, CRL
https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/CRL:CA:MSNT%257Cid=%2522%2522%257Cname/2023-10-01,2024-08-31/WTEQ::value?fitToS

  0%|          | 0/839 [00:00<?, ?it/s]

Start retrieving data for Adin Mtn, 301_CA_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/301:CA:SNTL%7Cid=%22%22%7Cname/2023-10-01,2024-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Agua Canyon, 907_UT_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/907:UT:SNTL%7Cid=%22%22%7Cname/2023-10-01,2024-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Albro Lake, 916_MT_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/916:MT:SNTL%7Cid=%22%22%7Cname/2023-10-01,2024-08-31/WTEQ::value?fitToScreen=false
Start retrieving data for Alexander Lake, 1267_AK_SNTL using https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/1267:AK:SNTL%7Cid=%22%22%7Cname/2023-10-01,2024-08-

# Code for generating ML dataframe using nearest in situ monitoring sites

In [ ]:
ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()

import GeoDF
#output_res = 1000


# GeoDF used to create a dataframe for ML model development. Its function is to connect in situ observations to gridded locations
for WY in WY_list:
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"

    if os.path.isdir(path) == True:
        print(WY)
        #load snotel meta location data, use haversive function
        GeoDF.fetch_snotel_sites_for_cellids(WY, output_res) # Using known up to date sites

        # Get geophysical attributes for each site, need to see how to add output resolution
        gdf = GeoDF.GeoSpatial(WY, output_res)

        #use geodataframe with lat/long meta of all sites to determine slope, aspect, and elevation
        metadf = GeoDF.extract_terrain_data_threaded(gdf, WY, output_res)
    else:
        print(f"No ASO data for {WY}")




2013
Loading all Geospatial prediction/observation files and concatenating into one dataframe


  0%|          | 0/6 [00:00<?, ?it/s]

Identifying unique sites to create geophysical information dataframe
converting to geodataframe
Processing snotel geometry
Calculating haversine distance for 5946 locations to in situ OBS, and saving cell-obs relationships in dictionary


  0%|          | 0/5946 [00:00<?, ?it/s]

Saving nearest SNOTEL in 2013 for each cell id in a pkl file
Loading geospatial data for 2013
Converting to geodataframe
Calculating dataframe bounding box
-121 36 -118 40
Retrieving Copernicus 90m DEM tiles


...

  0%|          | 0/80084 [00:00<?, ?it/s]

Saving nearest SNOTEL in 2015 for each cell id in a pkl file
Loading geospatial data for 2015
Converting to geodataframe
Calculating dataframe bounding box
-121 35 -105 40
Retrieving Copernicus 90m DEM tiles


  0%|          | 0/102 [00:00<?, ?it/s]

There are 102 tiles in the region
Determining Grid Cell Spatial Features


  0%|          | 0/80084 [00:00<?, ?it/s]

## Connect Snotel to each ASO obs


In [ ]:
ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()

import Obs_to_DF

#Connect nearest snotel observations with ASO data, makes a parquet file for each date  -  test to see if this works - need to just load the SNOTEL file, not collect them as in the function
for WY in WY_list:
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"

    if os.path.isdir(path) == True:
        print(WY)
        dates = []
        manual = False
        Obs_to_DF.Nearest_Snotel_2_obs_MultiProcess(WY, output_res, manual, dates) 
    else:
        print(f"No ASO data for {WY}")


In [ ]:
ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()
import GeoDF

#output_res = 1000

#Connect cell ids with ASO obs and snotel obs to geospatial features
for WY in WY_list:
    #path = f"{HOME}/SWEMLv2.0/data/ASO/{region}/{output_res}M_SWE_parquet"
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(WY)
        GeoDF.add_geospatial_threaded(WY, output_res)
    else:
        print(f"No ASO data for {region}")

# Get NASA VIIRS fraction snow covered area for each location 

* Make sure the code grabs all dates for each region, may have to run multiple times
* run until "No granules found for DATE, requesting data from NSIDC..." no longer occurs


In [13]:
get_VIIRS_SCA.get_VIIRS_from_AWS()

In [ ]:
ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()

import get_VIIRS_SCA
#output_res = 1000
threshold = 20

#check to see if the VIIRS data is available locally, if not, get from CIROH AWS - I think all of this data is for the incorrect year...
#get_VIIRS_SCA.get_VIIRS_from_AWS()

#region_list = ['Southwest']


#Connect VIIRS data to dataframes
for WY in WY_list:
    # path = f"{HOME}/SWEMLv2.0/data/ASO/{region}/{output_res}M_SWE_parquet"
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(WY)
        get_VIIRS_SCA.augment_SCA_multiprocessing(WY, output_res, threshold)
    else:
        print(f"No ASO data for {WY}")


In [5]:
WY_list = ['2019', '2020', '2021', '2022', '2023', '2024'] #'2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024' - Trying the first bit to prove concept, then can expand


In [ ]:
ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()

import get_Precip

'''
note*, if using python > 3.9, you will likely need to change the ee package to from io import StringIO
sometimes there will be an ASO file that is inproperly named in the ASO yr folder, it typically also makes a ,ipynb checkpoint that crash the code
'''

# import os
# HOME = os.path.expanduser('~')

#gets precipitation for each location, accumulates it through the water year
#This step could be made much more efficient by collecting all of the tiles in one step, then multiprocessing later

#set start/end date for a water year
years = [2019, 2020, 2021, 2022, 2023, 2024] #[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
#output_res = 1000
threshold = 20

for WY in WY_list:
    # path = f"{HOME}/SWEMLv2.0/data/ASO/{region}/{output_res}M_SWE_parquet"
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"

    if os.path.isdir(path) == True:
        print(WY)
        get_Precip.get_precip_threaded(WY, output_res, years)
    else:
        print(f"No ASO data for {WY}")

    #Connect precipitation to processed DFs
    get_Precip.Make_Precip_DF(WY, output_res, threshold)


## Add seasonality metrics to the dataframe

In [7]:
WY_list = ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'] #'2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024' - Trying the first bit to prove concept, then can expand


In [8]:
ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()

import get_Seasonality

#output_res = 1000
threshold = 20
for WY in WY_list:
    #process snotel sites to make "snow hydrograph features" to determine above/below average WY conditions
    get_Seasonality.seasonal_snotel()


    #get the Day of season metric for each dataframe
    get_Seasonality.add_Seasonality(WY, output_res, threshold)

Adding Day of Season, seasonal nearest monitoring site averages, and seasonal nearest monitoring site relationship to averages to all 2013 dataframes...


  0%|          | 0/6 [00:00<?, ?it/s]


100%|██████████| 1488/1488 [00:00<00:00, 2248.21it/s]

100%|██████████| 1488/1488 [00:00<00:00, 2099.69it/s]


Adding Day of Season, seasonal nearest monitoring site averages, and seasonal nearest monitoring site relationship to averages to all 2014 dataframes...


  0%|          | 0/18 [00:00<?, ?it/s]

...

  0%|          | 0/60 [00:00<?, ?it/s]


100%|██████████| 474/474 [00:05<00:00, 80.22it/s]

100%|██████████| 474/474 [00:06<00:00, 71.45it/s]s]

100%|██████████| 906/906 [00:04<00:00, 188.45it/s]]

100%|██████████| 899/899 [00:07<00:00, 126.89it/s]]

100%|██████████| 903/903 [00:06<00:00, 134.39it/s]s]

100%|██████████| 1318/1318 [00:05<00:00, 248.29it/s]

100%|██████████| 896/896 [00:03<00:00, 267.06it/s]]

100%|██████████| 1370/1370 [00:06<00:00, 203.87it/s]

100%|██████████| 1638/1638 [00:05<00:00, 275.15it/s]

100%|██████████| 1638/1638 [00:05<00:00, 313.61it/s]

100%|██████████| 1792/1792 [00:03<00:00, 471.77it/s]

100%|██████████| 1792/1792 [00:04<00:00, 412.19it/s]

 42%|████▏     | 2171/5216 [00:03<00:06, 463.66it/s]
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window

Adding Day of Season, seasonal nearest monitoring site averages, and seasonal nearest monitoring site relationship to averages to all 2024 dataframes...


  0%|          | 0/43 [00:00<?, ?it/s]


100%|██████████| 903/903 [00:01<00:00, 475.36it/s]]

100%|██████████| 896/896 [00:03<00:00, 258.19it/s]]

100%|██████████| 1118/1118 [00:02<00:00, 372.93it/s]

100%|██████████| 1318/1318 [00:03<00:00, 373.93it/s]

100%|██████████| 1118/1118 [00:04<00:00, 272.89it/s][A

100%|██████████| 1083/1083 [00:04<00:00, 243.02it/s]

100%|██████████| 896/896 [00:03<00:00, 237.38it/s]s]

100%|██████████| 1792/1792 [00:03<00:00, 498.11it/s]

100%|██████████| 1961/1961 [00:03<00:00, 513.62it/s]

100%|██████████| 1873/1873 [00:03<00:00, 482.72it/s]

100%|██████████| 1849/1849 [00:03<00:00, 521.79it/s]

100%|██████████| 1873/1873 [00:03<00:00, 574.40it/s]

100%|██████████| 1820/1820 [00:03<00:00, 479.83it/s]]

100%|██████████| 1621/1621 [00:04<00:00, 357.73it/s]

100%|██████████| 2234/2234 [00:03<00:00, 606.85it/s] 

100%|██████████| 2453/2453 [00:03<00:00, 648.30it/s]]

100%|██████████| 2613/2613 [00:03<00:00, 702.45it/s]

100%|██████████| 2210/2210 [00:03<00:00, 612.17it/s]]

100%|██████████| 2235/2

# Use Sturm's snow classification as features within model framework

Using the originally created env, it looks like the rasterio package does not contain the correct ECS driver. Trying to address this with conda install conda-forge::rasterio in my SWEML_310 env from the shell in CHPC

In [9]:
ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()

import sturm_processer as stpro
#outputres =1000
thres = 20

#download sturm data
stpro.get_Sturm_data()

for WY in WY_list:
    input_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Seasonality_PrecipVIIRSGeoObsDFs/{thres}_fSCA_Thresh"
    sturm_file = f"{HOME}/data/SnowClassification/SnowClass_NA_300m_10.0arcsec_2021_v01.0.tif" #https://nsidc.org/data/nsidc-0768/versions/1
    output_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Sturm_Seasonality_PrecipVIIRSGeoObsDFs/{thres}_fSCA_Thresh"
    
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    stpro.process_sturm_data_for_files(input_directory, sturm_file, output_directory)

sturm data already downloaded
Sturm file bounds: BoundingBox(left=-180.0, bottom=0.0, right=-10.0, top=90.0)


Processing Parquet Files: 100%|██████████| 6/6 [00:11<00:00,  1.86s/it]


Sturm file bounds: BoundingBox(left=-180.0, bottom=0.0, right=-10.0, top=90.0)


Processing Parquet Files: 100%|██████████| 18/18 [00:33<00:00,  1.89s/it]


Sturm file bounds: BoundingBox(left=-180.0, bottom=0.0, right=-10.0, top=90.0)


...

Processing Parquet Files: 100%|██████████| 36/36 [01:08<00:00,  1.92s/it]


Sturm file bounds: BoundingBox(left=-180.0, bottom=0.0, right=-10.0, top=90.0)


Processing Parquet Files: 100%|██████████| 60/60 [02:01<00:00,  2.02s/it]


Sturm file bounds: BoundingBox(left=-180.0, bottom=0.0, right=-10.0, top=90.0)


Processing Parquet Files: 100%|██████████| 43/43 [01:26<00:00,  2.01s/it]


# Add vegetation data to the dataframe from the North American land Cover Management Systemoutput_path

In [10]:
ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()

import vegetation_processer as vegpro
import os

#get data
url = "http://www.cec.org/files/atlas_layers/1_terrestrial_ecosystems/1_01_0_land_cover_2020_30m/usa_land_cover_2020v2_30m_tif.zip"
output_path = f"{HOME}/data/LandCover/"
file = "usa_land_cover_2020v2_30m_tif.zip" 
vegpro.get_data(url, output_path, file)
#unzip the file is not already done
#vegpro.unzip_LC_data(output_path, file)
#output = 1000 

for WY in WY_list:
    input_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Sturm_Seasonality_PrecipVIIRSGeoObsDFs/20_fSCA_Thresh"
    vegetation_file = f"{HOME}/data/LandCover/usa_land_cover_2020v2_30m_tif/USA_NALCMS_landcover_2020v2_30m/data/USA_NALCMS_landcover_2020v2_30m.tif"
    output_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Vegetation_Sturm_Seasonality_PrecipVIIRSGeoObsDFs/20_fSCA_Thresh"
    
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    vegpro.process_vegetation_data_for_files(input_directory, vegetation_file, output_directory)

Landcover data already downloaded
Vegetation file bounds: BoundingBox(left=-2043060.0, bottom=-2113150.0, right=2529600.0, top=732440.0)
Vegetation CRS: PROJCS["WGS_1984_Lambert_Azimuthal_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-100],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


Processing Parquet Files: 100%|██████████| 6/6 [01:39<00:00, 16.66s/it]


Vegetation file bounds: BoundingBox(left=-2043060.0, bottom=-2113150.0, right=2529600.0, top=732440.0)
Vegetation CRS: PROJCS["WGS_1984_Lambert_Azimuthal_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-100],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


Processing Parquet Files: 100%|██████████| 18/18 [04:59<00:00, 16.65s/it]


Vegetation file bounds: BoundingBox(left=-2043060.0, bottom=-2113150.0, right=2529600.0, top=732440.0)
Vegetation CRS: PROJCS["WGS_1984_Lambert_Azimuthal_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-100],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


...

Processing Parquet Files: 100%|██████████| 36/36 [10:05<00:00, 16.83s/it]


Vegetation file bounds: BoundingBox(left=-2043060.0, bottom=-2113150.0, right=2529600.0, top=732440.0)
Vegetation CRS: PROJCS["WGS_1984_Lambert_Azimuthal_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-100],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


Processing Parquet Files: 100%|██████████| 60/60 [16:51<00:00, 16.86s/it]


Vegetation file bounds: BoundingBox(left=-2043060.0, bottom=-2113150.0, right=2529600.0, top=732440.0)
Vegetation CRS: PROJCS["WGS_1984_Lambert_Azimuthal_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-100],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


Processing Parquet Files: 100%|██████████| 43/43 [12:04<00:00, 16.84s/it]


## Next steps
* Explore why errors in precip sites above
* add in situ obs - seasonality based on the historical neareste x monitoring stations - like a historical average to-date swe value unit hydrograph based on the day of year? This will include a historical time of year of normal swe value and a swe value of year compared to normal
* albedo metric


In [ ]:
import pandas as pd

HOME = os.path.expanduser('~')
region = 'Southwest'
output_res = '300'

dfpath = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution"

SWmeta = pd.read_parquet(f"{dfpath}/{region}_metadata.parquet")

import UpdateDataFrame

#need to update the topographic features for every dataframe
output_res = '300'
training_cats = ['Obsdf']
fSCA = '' #'20_fSCA_Thresh'


for training_cat in training_cats:
    print(training_cat)

    for region in region_list:
        print(region)
        dfpath = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution"
        #file to be used to updated training DF
        updatefile = pd.read_parquet(f"{dfpath}/{region}_metadata.parquet")


        #Update Dataframe
        UpdateDataFrame.updateTrainingDF(region, output_res, training_cat, fSCA, updatefile)

trainfile = pd.read_parquet(f"{dfpath}/{training_cat}/{fSCA}/Sturm_Season_Precip_VIIRS_GeoObsDF_20150406.parquet")

import matplotlib.pyplot as plt
import geopandas as gpd

from mpl_toolkits.axes_grid1 import make_axes_locatable

def SpatialAnalysis(EvalDF):
    #Convert to a geopandas DF
    Pred_Geo = gpd.GeoDataFrame(EvalDF, geometry = gpd.points_from_xy(EvalDF.cen_lon, EvalDF.cen_lat))

    Pred_Geo.plot(column='Elevation_m',
                  legend=False,
                )
    
SpatialAnalysis(trainfile)